In [1]:
import pickle

In [2]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords
from pymystem3 import Mystem
import re
from collections import Counter

In [3]:
import pandas as pd
import numpy as np

In [4]:
title_lengths = dict()
with open('./lengths.title', 'rb') as fin:
    title_lengths = pickle.load(fin)
    
text_lengths = dict()
with open('./lengths.text', 'rb') as fin:
    text_lengths = pickle.load(fin)

In [5]:
title_avgdl = np.mean(list(title_lengths.values()))
text_avgdl = np.mean(list(text_lengths.values()))

In [6]:
# df_title = dict()
# df_text = dict()
df_total = dict()

# with open('./title.df', 'rb') as fin:
#     df_title = pickle.load(fin)

# with open('./text.df', 'rb') as fin:
#     df_text = pickle.load(fin)

with open('./total.df', 'rb') as fin:
    df_total = pickle.load(fin)

In [7]:
lemmatizer = Mystem()
pattern = re.compile('\d+|[^\W\d]+')
stemmer = SnowballStemmer('russian', ignore_stopwords=True)

In [8]:
queries = []
with open('queries.final', 'r') as fin:
    for line in fin:
        q = line.split('\t')[1]
        queries.append(list(map(lambda x: stemmer.stem(lemmatizer.lemmatize(x)[0]), pattern.findall(q))))

In [9]:
sample = pd.read_csv('./sample.csv', sep=',')

In [10]:
q2docs = []
for i in range(6311):
    q2docs.append(list(sample[sample['QueryId'] == i]['DocumentId']))

In [38]:
# tf-idf

In [42]:
marks = []
weights = [0.5, 1.0]
N = 582167
for q, i in zip(queries, range(len(queries))):
    marks.append([])
    
    if (i + 1) % 100 == 0:
        print('processed', i + 1)
    
    for doc_id in q2docs[i]:
        idx = str(doc_id)
        
        fq_title = dict()
        fq_text = dict()
        with open('./frequences/' + idx + '.title', 'rb') as fin:
            fq_title = pickle.load(fin)
        with open('./frequences/' + idx + '.text', 'rb') as fin:
            fq_text = pickle.load(fin)
        
        rank = 0.0
        for w in q:
            if not w in df_total:
                continue

            if df_total[w] > N // 2:
                continue
                
            coeff = np.log((N - df_total[w] + 0.5) / (df_total[w] + 0.5))
            mult = 0.0
            
            if w in fq_title:
                mult += weights[0] * fq_title[w] * 1.0 / title_lengths[idx]
                
            if w in fq_text:
                mult += weights[1] * fq_text[w] * 1.0 / text_lengths[idx]
            rank += coeff * mult
            
        marks[-1].append(rank)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
processed 1700
processed 1800
processed 1900
processed 2000
processed 2100
processed 2200
processed 2300
processed 2400
processed 2500
processed 2600
processed 2700
processed 2800
processed 2900
processed 3000
processed 3100
processed 3200
processed 3300
processed 3400
processed 3500
processed 3600
processed 3700
processed 3800
processed 3900
processed 4000
processed 4100
processed 4200
processed 4300
processed 4400
processed 4500
processed 4600
processed 4700
processed 4800
processed 4900
processed 5000
processed 5100
processed 5200
processed 5300
processed 5400
processed 5500
processed 5600
processed 5700
processed 5800
processed 5900
processed 6000
processed 6100
processed 6200
processed 6300


In [ ]:
# bm25

In [46]:
marks = []
weights = [3.0, 1.0]
N = 582167
k1 = 1.2
b = 0.75

for q, i in zip(queries, range(len(queries))):
    marks.append([])
    
    if (i + 1) % 100 == 0:
        print('processed', i + 1)
    
    for doc_id in q2docs[i]:
        idx = str(doc_id)
        
        fq_title = dict()
        fq_text = dict()
        with open('./frequences/' + idx + '.title', 'rb') as fin:
            fq_title = pickle.load(fin)
        with open('./frequences/' + idx + '.text', 'rb') as fin:
            fq_text = pickle.load(fin)
        
        rank = 0.0
        for w in q:
            if not w in df_total:
                continue

            if df_total[w] > N // 2:
                continue
                
            coeff = np.log((N - df_total[w] + 0.5) / (df_total[w] + 0.5))
            mult = 0.0
            
            if w in fq_title:
                mult += weights[0] * fq_title[w] * (k1 + 1.0) / (fq_title[w]+k1*(1-b+b*title_lengths[idx]/title_avgdl))
                
            if w in fq_text:
                mult += weights[1] * fq_text[w] * (k1 + 1.0) / (fq_text[w]+k1*(1-b+b*text_lengths[idx]/text_avgdl))
            rank += coeff * mult
            
        marks[-1].append(rank)

processed 100
processed 200
processed 300
processed 400
processed 500
processed 600
processed 700
processed 800
processed 900
processed 1000
processed 1100
processed 1200
processed 1300
processed 1400
processed 1500
processed 1600
processed 1700
processed 1800
processed 1900
processed 2000
processed 2100
processed 2200
processed 2300
processed 2400
processed 2500
processed 2600
processed 2700
processed 2800
processed 2900
processed 3000
processed 3100
processed 3200
processed 3300
processed 3400
processed 3500
processed 3600
processed 3700
processed 3800
processed 3900
processed 4000
processed 4100
processed 4200
processed 4300
processed 4400
processed 4500
processed 4600
processed 4700
processed 4800
processed 4900
processed 5000
processed 5100
processed 5200
processed 5300
processed 5400
processed 5500
processed 5600
processed 5700
processed 5800
processed 5900
processed 6000
processed 6100
processed 6200
processed 6300


In [47]:
res = []
for arr, i in zip(marks, range(6311)):
    for idx in np.array(q2docs[i])[np.argsort(arr)[-1:-6:-1]]:
        res.append([i, idx])

In [48]:
res = np.array(res)

In [49]:
result = pd.DataFrame(columns=['QueryId', 'DocumentId'])

In [50]:
result['QueryId'] = res[:, 0]
result['DocumentId'] = res[:, 1]

In [51]:
result.to_csv('./result.csv', index=False, sep=',')

In [35]:
result.head()

,QueryId,DocumentId
0,0,49847
1,0,563677
2,0,461894
3,0,64459
4,0,53265


In [53]:
result.head()

,QueryId,DocumentId
0,0,461894
1,0,461895
2,0,563677
3,0,46868
4,0,113720


In [56]:
!kaggle competitions submit -c ranking-long-tail-queries-ts-spring-2018 -f result.csv -m 'bm25'

^C
Traceback (most recent call last):
  File "/usr/local/lib/python3.5/dist-packages/urllib3/connectionpool.py", line 380, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 9, in <module>
    load_entry_point('kaggle==1.3.5', 'console_scripts', 'kaggle')()
  File "/usr/local/lib/python3.5/dist-packages/kaggle/cli.py", line 37, in main
    out = args.func(**command_args)
  File "/usr/local/lib/python3.5/dist-packages/kaggle/api/kaggle_api_extended.py", line 196, in competition_submit
    last_modified_date_utc=int(os.path.getmtime(file) * 1000)))
  File "/usr/local/lib/python3.5/dist-packages/kaggle/api/kaggle_api.py", line 773, in competitions_submissions_url_with_http_info
    collection_formats=collection_formats)
  File "/usr/local/lib/python3.5/dist